### Diseño de Agentes Inteligentes
### Tarea 2. Agentes Solucionadores de Problemas (PSA) y búsqueda

*Integrantes del Equipo 4* 
1. Ricardo Kaleb Flores Alfonso     A01198716
2. Sebastián Miramontes Soto        A01285296
3. Raúl Correa Ocañas               A01722401


#### Cleanup Puzzle

Cleanup Puzzle es un juego de mesa que requiere que limpies el tablero. Para ello, deberás hacer clic en las fichas de un tablero tipo cuadrícula. Cada ficha puede estar vacía o contener una bola. El mosaico en el que haga clic no se verá afectado, pero sus vecinos directos (horizontal y verticalmente) se invertirán (si el mosaico contiene una bola, se vaciará y viceversa). El objetivo es determinar una secuencia de clics que elimine todas las bolas del tablero.

El juego, que puedes encontrar y jugar en el enlace https://www.mathsisfun.com/games/cleanup-puzzle.html , consiste en un tablero de 11x11 que se juega en una secuencia de etapas de complejidad incrementada. La complejidad de un escenario está relacionada con la cantidad de bolas en el tablero y sus ubicaciones. En el juego en línea, hay un límite de clics definido para cada etapa y el juego termina cuando no puede encontrar una secuencia de clics lo suficientemente pequeña para limpiar el tablero.
Ejemplos de diferentes etapas del juego en línea:

![alt text](ejemplo1.png)

![alt text](ejemplo2.png)


In [11]:
import numpy as np
from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer

In [12]:
## Se crean los estados iniciales por dificultad
init = ((0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

init2 =((0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0),
        (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0),
        (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1),
        (0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0),
        (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0))

init3 =((0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0),
        (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0),
        (1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0),
        (1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0),
        (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0),
        (1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
        (0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0))

In [13]:
#Se suma cuantas bolitas hay alrededor de la coordenada otorgada
def adjacents(state, h,w):
        try:
            x = state[h][w+1]+ state[h][w-1]+state[h+1][w]+state[h-1][w]
            return x
        except:
            try:
                x = state[h][w+1]+state[h][w-1]+state[h+1][w]
                return x
            except:
                try:
                    x = state[h][w+1]+state[h][w-1]+state[h-1][w]
                    return x
                except:
                    try:
                        x = state[h][w+1]+state[h+1][w]+state[h-1][w]
                        return x
                    except:
                        try:
                            x = state[h][w-1]+state[h+1][w]+state[h-1][w]
                            return x
                        except:
                                try:
                                    x = state[h][w+1]+state[h+1][w]
                                    return x
                                except:
                                    try:
                                        x = state[h][w+1]+state[h-1][w]
                                        return x
                                    except:
                                        try:
                                            x = state[h][w-1]+state[h+1][w]
                                            return x
                                        except:
                                            try:
                                                x = state[h][w-1]+state[h-1][w]
                                                return x
                                            except:
                                                z=1

In [14]:
class Mine(SearchProblem):

    def init(self, initial):
        self.initial = initial
        SearchProblem.__init__(self,initial)
        self.goal_state = 0

    def actions(self,state):
      #Para las acciones se eliminan aquellas que agregan 4 o 3 bolitas al tablero y se les asigna un costo dependiendo si estan cerca de la orilla o tienen bolitas alrededor
        actions = []
        child_state = [list(row) for row in state]
        for i in range(11):
            for j in range(11):
                if adjacents(child_state,i,j) == 4:
                  actions.append([i,j,0])
                elif adjacents(child_state,i,j) == 3:
                  cost = 1
                  if i - 1== -1:
                    cost = cost -1
                  if j -1 == -1:
                    cost = cost -1
                  if j+1 > len(state[0]):
                    cost = cost -1
                  if i+1 > len(state):
                    cost = cost -1
                  actions.append([i,j,cost])

                elif adjacents(child_state,i,j) == 2:
                  
                  cost = 30
                  if i - 1== -1:
                    cost = cost -15
                  if j -1 == -1:
                    cost = cost -15
                  if j+1 > len(state[0]):
                    cost = cost -15
                  if i+1 > len(state):
                    cost = cost -15
                  actions.append([i,j,cost])
        return actions


    def result(self, state, action):
      #Para los resultados se usa not y de esta manera se voltean los valores de las bolitas alrededor de la coordenada dada
        i = action[0]
        j = action[1]
        child_state = [list(row) for row in state]

        if i - 1 >= 0:
            child_state[i - 1][j] = not child_state[i - 1][j]
        if j - 1 >= 0:
            child_state[i][j - 1] = not child_state[i][j - 1]
        if j + 1 < len(state[0]):
            child_state[i][j + 1] = not child_state[i][j + 1]
        if i + 1 < len(state):
            child_state[i + 1][j] = not child_state[i + 1][j]
        return tuple([tuple(row) for row in child_state])

    def is_goal(self,state):
      #Cuando la suma de las bolitas es 0 se ha llegado a la meta
        found_goal = np.sum(state) == 0
        return found_goal
    def cost(self,state1,action,state2):
      #En esa posición esta guardado el costo de la acción
          return action[2]

# Función para mostrar el resultado de la búsqueda
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
              print('Configuración inicial')
            elif i == len(result.path()) - 1:
              print(i, action)
              print('¡Meta lograda con costo = ', result.cost,'!')
            else:
              print(i, action)

            print('  ', state)
    else:
      print('Mala configuración del problema')







In [15]:
# Crea un PSA y lo resuelve con la búsqueda primero en anchura
my_viewer = BaseViewer()
result = uniform_cost(Mine(init3),graph_search=True, viewer=my_viewer)
display(result)

if my_viewer != None:
    print('Stats:')
    print(my_viewer.stats)

print()
print('>> Costo uniforme <<')
display(result)

MemoryError: 